In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
import openpyxl
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
columns = ["comment"]
df = pd.read_excel(r'C:\Users\Hp\TUGAS AKHIR 2\YOUTUBE_INDIHOME\data_mentah\youtube_indihome_1.xlsx', usecols=columns)
df.head(2000)

comment
0    Bang kalo misalnya kita sdh daftar tingal nung...
1            Apa setiap wilayah berbeda-beda harganya?
2                             Ikut nyimak kawan,sukses
3                                Klo ke desa bisa gak?
4     Bang nunggu verivikasi telponnya berapa lama ya?
..                                                 ...
112  Bang itu speednya k uba setela pembayaran atw ...
113           bang add on movin itu gratis apa bayar ?
114  bang kl sy upgrade speed setelah pembayaran bu...
115                        Thanks gan, sangat membantu
116                    Tiap x beli jawaban nya c saldo

[117 rows x 1 columns]

In [3]:
df.comment.duplicated().sum()

37

In [4]:
df.drop_duplicates(subset="comment", keep=False, inplace=True)
df.head(2000)

comment
0    Bang kalo misalnya kita sdh daftar tingal nung...
1            Apa setiap wilayah berbeda-beda harganya?
2                             Ikut nyimak kawan,sukses
3                                Klo ke desa bisa gak?
4     Bang nunggu verivikasi telponnya berapa lama ya?
5    Susah kalau lewat HP.. Saya lgsg pergi ke kant...
6    Saya mau pasang tapi tidak ada tiang di depan ...
33                                           2022 dunk
34   Bang btw mau nanya pasang indohome itu hargany...
35   bang mau nanya ni saya waktu pasang bayar pert...
36                    Ada paket untuk usaha RT/RW net?
37                                                 NaN
38                                 Lemes amat suaranya
67   Di apk myindihome ada yang 240rb/ bulan itu as...
68   Mau nanya itu kalau tv dirumahnya msh tv lama ...
69                      Useetv itu nyedot kuota tidak?
70   KLLo pasang indihome baru apakah dapet STB ka?...
71      Kira2 sekitar 200k ada kah di wilayah surabaya
72                                5 hari yg lalu hadir
73                  Kereeen. Salam kenal dan sukses ya
74                         banyak bacot penjelasannya,
75   Kak kalo dari paket internet dan tv kyk kakak ...
76   Bang, kalo upgrade via aplikasi fup kuotanya n...
77                                            kga bisa
78          Kalo bisa. Minta tolong jangan bisik2 bang
79   Ah ampe 4x nelpon ke 147 ampe skrg ga berubah ...
80                  Kok nambahnya 70 rb yak bukan 20rb
81   Kalau gantipaket/cari lebih murah bisa g bang ...
88             Batas FUP 20mbps berapa bang? Sama 350?
89   Kok gagal .. nomer layananan anda tidak kompot...
104  mas saya kok tdk bisa speed on dimaind kenapa ...
105            Upgarad speed sy yg 20Mbps 40rebu uuyy😖
106  Saya langganan paket internet 10mb + telepon d...
107  Itu bang selama sudah di upgrade menjadi 20 mb...
108  saya upgrade 10 mbps 20 mbs apa benner cuma na...
109  setiap mau upgrade speed dari 10mbps ke 20mbps...
110               klo 100mbps apa hrs ke plaza telkom?
111  Bang saya kan berlangganan paket phoenix 10 mb...
112  Bang itu speednya k uba setela pembayaran atw ...
113           bang add on movin itu gratis apa bayar ?
114  bang kl sy upgrade speed setelah pembayaran bu...
115                        Thanks gan, sangat membantu
116                    Tiap x beli jawaban nya c saldo

In [5]:
#case folding
df['comment'] = df['comment'].str.lower()
df.head(2000)

comment
0    bang kalo misalnya kita sdh daftar tingal nung...
1            apa setiap wilayah berbeda-beda harganya?
2                             ikut nyimak kawan,sukses
3                                klo ke desa bisa gak?
4     bang nunggu verivikasi telponnya berapa lama ya?
5    susah kalau lewat hp.. saya lgsg pergi ke kant...
6    saya mau pasang tapi tidak ada tiang di depan ...
33                                           2022 dunk
34   bang btw mau nanya pasang indohome itu hargany...
35   bang mau nanya ni saya waktu pasang bayar pert...
36                    ada paket untuk usaha rt/rw net?
37                                                 NaN
38                                 lemes amat suaranya
67   di apk myindihome ada yang 240rb/ bulan itu as...
68   mau nanya itu kalau tv dirumahnya msh tv lama ...
69                      useetv itu nyedot kuota tidak?
70   kllo pasang indihome baru apakah dapet stb ka?...
71      kira2 sekitar 200k ada kah di wilayah surabaya
72                                5 hari yg lalu hadir
73                  kereeen. salam kenal dan sukses ya
74                         banyak bacot penjelasannya,
75   kak kalo dari paket internet dan tv kyk kakak ...
76   bang, kalo upgrade via aplikasi fup kuotanya n...
77                                            kga bisa
78          kalo bisa. minta tolong jangan bisik2 bang
79   ah ampe 4x nelpon ke 147 ampe skrg ga berubah ...
80                  kok nambahnya 70 rb yak bukan 20rb
81   kalau gantipaket/cari lebih murah bisa g bang ...
88             batas fup 20mbps berapa bang? sama 350?
89   kok gagal .. nomer layananan anda tidak kompot...
104  mas saya kok tdk bisa speed on dimaind kenapa ...
105            upgarad speed sy yg 20mbps 40rebu uuyy😖
106  saya langganan paket internet 10mb + telepon d...
107  itu bang selama sudah di upgrade menjadi 20 mb...
108  saya upgrade 10 mbps 20 mbs apa benner cuma na...
109  setiap mau upgrade speed dari 10mbps ke 20mbps...
110               klo 100mbps apa hrs ke plaza telkom?
111  bang saya kan berlangganan paket phoenix 10 mb...
112  bang itu speednya k uba setela pembayaran atw ...
113           bang add on movin itu gratis apa bayar ?
114  bang kl sy upgrade speed setelah pembayaran bu...
115                        thanks gan, sangat membantu
116                    tiap x beli jawaban nya c saldo

In [6]:
#cleaning text
import nltk
import string
import re
nltk.download('punkt')

#import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#proses cleaning text
def remove_special_tweet(text):
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", "", str(text)).split())
    text = re.sub(r'[^a-zA-Z0-9]', ' ', str(text))
    text = re.sub(r'\b\w{1,2}\b', ' ', str(text))
    text = re.sub(r'\s\s+',' ',  str(text))
    text = re.sub(r'\d+', ' ', str(text))
    text = re.sub(r'\b[a-zA-Z]\b', ' ', str(text))
    text = re.sub(r'/(?:https?|ftp):\/\/[\n\S]+/g, ''', ' ', str(text))
    text = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', str(text))
    text = re.sub(r'[0]', r' ', str(text))
    return text
    return df
df['cleaning_tweet'] = df['comment'].apply(remove_special_tweet)
df.head(2000)

comment  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda-beda harganya?   
2                             ikut nyimak kawan,sukses   
3                                klo ke desa bisa gak?   
4     bang nunggu verivikasi telponnya berapa lama ya?   
5    susah kalau lewat hp.. saya lgsg pergi ke kant...   
6    saya mau pasang tapi tidak ada tiang di depan ...   
33                                           2022 dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya ni saya waktu pasang bayar pert...   
36                    ada paket untuk usaha rt/rw net?   
37                                                 NaN   
38                                 lemes amat suaranya   
67   di apk myindihome ada yang 240rb/ bulan itu as...   
68   mau nanya itu kalau tv dirumahnya msh tv lama ...   
69                      useetv itu nyedot kuota tidak?   
70   kllo pasang indihome baru apakah dapet stb ka?...   
71      kira2 sekitar 200k ada kah di wilayah surabaya   
72                                5 hari yg lalu hadir   
73                  kereeen. salam kenal dan sukses ya   
74                         banyak bacot penjelasannya,   
75   kak kalo dari paket internet dan tv kyk kakak ...   
76   bang, kalo upgrade via aplikasi fup kuotanya n...   
77                                            kga bisa   
78          kalo bisa. minta tolong jangan bisik2 bang   
79   ah ampe 4x nelpon ke 147 ampe skrg ga berubah ...   
80                  kok nambahnya 70 rb yak bukan 20rb   
81   kalau gantipaket/cari lebih murah bisa g bang ...   
88             batas fup 20mbps berapa bang? sama 350?   
89   kok gagal .. nomer layananan anda tidak kompot...   
104  mas saya kok tdk bisa speed on dimaind kenapa ...   
105            upgarad speed sy yg 20mbps 40rebu uuyy😖   
106  saya langganan paket internet 10mb + telepon d...   
107  itu bang selama sudah di upgrade menjadi 20 mb...   
108  saya upgrade 10 mbps 20 mbs apa benner cuma na...   
109  setiap mau upgrade speed dari 10mbps ke 20mbps...   
110               klo 100mbps apa hrs ke plaza telkom?   
111  bang saya kan berlangganan paket phoenix 10 mb...   
112  bang itu speednya k uba setela pembayaran atw ...   
113           bang add on movin itu gratis apa bayar ?   
114  bang kl sy upgrade speed setelah pembayaran bu...   
115                        thanks gan, sangat membantu   
116                    tiap x beli jawaban nya c saldo   

                                        cleaning_tweet  
0    bang kalo misalnya kita sdh daftar tingal nung...  
1            apa setiap wilayah berbeda beda harganya   
2                             ikut nyimak kawan sukses  
3                                   klo desa bisa gak   
4        bang nunggu verivikasi telponnya berapa lama   
5          susah kalau lewat saya lgsg pergi kantornya  
6    saya mau pasang tapi tidak ada tiang depan rum...  
33                                                dunk  
34   bang btw mau nanya pasang indohome itu hargany...  
35   bang mau nanya saya waktu pasang bayar pertma ...  
36                          ada paket untuk usaha net   
37                                                 nan  
38                                 lemes amat suaranya  
67    apk myindihome ada yang  rb bulan itu asli ng...  
68   mau nanya itu kalau dirumahnya msh lama tabung...  
69                      useetv itu nyedot kuota tidak   
70   kllo pasang indihome baru apakah dapet stb say...  
71           kira  sekitar    ada kah wilayah surabaya  
72                                     hari lalu hadir  
73                     kereeen salam kenal dan sukses   
74                         banyak bacot penjelasannya   
75   kak kalo dari paket internet dan kyk kakak mbp...  
76   bang kalo upgrade via aplikasi fup kuotanya na...  
77                                            kga bisa  
78           kalo bisa minta tolong jangan bisik  bang  
79    ampe n

In [8]:
#proses tokenisasi
def tokenisasi(text):
    text = re.split('\W+', str(text))
    return text
    return df
df['tokenisasi'] = df['cleaning_tweet'].apply(tokenisasi)
df.head(2000)

comment  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda-beda harganya?   
2                             ikut nyimak kawan,sukses   
3                                klo ke desa bisa gak?   
4     bang nunggu verivikasi telponnya berapa lama ya?   
5    susah kalau lewat hp.. saya lgsg pergi ke kant...   
6    saya mau pasang tapi tidak ada tiang di depan ...   
33                                           2022 dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya ni saya waktu pasang bayar pert...   
36                    ada paket untuk usaha rt/rw net?   
37                                                 NaN   
38                                 lemes amat suaranya   
67   di apk myindihome ada yang 240rb/ bulan itu as...   
68   mau nanya itu kalau tv dirumahnya msh tv lama ...   
69                      useetv itu nyedot kuota tidak?   
70   kllo pasang indihome baru apakah dapet stb ka?...   
71      kira2 sekitar 200k ada kah di wilayah surabaya   
72                                5 hari yg lalu hadir   
73                  kereeen. salam kenal dan sukses ya   
74                         banyak bacot penjelasannya,   
75   kak kalo dari paket internet dan tv kyk kakak ...   
76   bang, kalo upgrade via aplikasi fup kuotanya n...   
77                                            kga bisa   
78          kalo bisa. minta tolong jangan bisik2 bang   
79   ah ampe 4x nelpon ke 147 ampe skrg ga berubah ...   
80                  kok nambahnya 70 rb yak bukan 20rb   
81   kalau gantipaket/cari lebih murah bisa g bang ...   
88             batas fup 20mbps berapa bang? sama 350?   
89   kok gagal .. nomer layananan anda tidak kompot...   
104  mas saya kok tdk bisa speed on dimaind kenapa ...   
105            upgarad speed sy yg 20mbps 40rebu uuyy😖   
106  saya langganan paket internet 10mb + telepon d...   
107  itu bang selama sudah di upgrade menjadi 20 mb...   
108  saya upgrade 10 mbps 20 mbs apa benner cuma na...   
109  setiap mau upgrade speed dari 10mbps ke 20mbps...   
110               klo 100mbps apa hrs ke plaza telkom?   
111  bang saya kan berlangganan paket phoenix 10 mb...   
112  bang itu speednya k uba setela pembayaran atw ...   
113           bang add on movin itu gratis apa bayar ?   
114  bang kl sy upgrade speed setelah pembayaran bu...   
115                        thanks gan, sangat membantu   
116                    tiap x beli jawaban nya c saldo   

                                        cleaning_tweet  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda beda harganya    
2                             ikut nyimak kawan sukses   
3                                   klo desa bisa gak    
4        bang nunggu verivikasi telponnya berapa lama    
5          susah kalau lewat saya lgsg pergi kantornya   
6    saya mau pasang tapi tidak ada tiang depan rum...   
33                                                dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya saya waktu pasang bayar pertma ...   
36                          ada paket untuk usaha net    
37                                                 nan   
38                                 lemes amat suaranya   
67    apk myindihome ada yang  rb bulan itu asli ng...   
68   mau nanya itu kalau dirumahnya msh lama tabung...   
69                      useetv itu nyedot kuota tidak    
70   kllo pasang indihome baru apakah dapet stb say...   
71           kira  sekitar    ada kah wilayah surabaya   
72                                     hari lalu hadir   
73                     kereeen salam kenal dan sukses    
74                         banyak bacot penjelasannya    
75   kak kalo dari paket internet dan kyk kakak mbp...   
76   bang kalo upgrade via aplikasi fup kuotanya na...   
77                                            kga bisa   
78           kalo bisa minta tolong jangan b

In [9]:
#import nltk stopword
from nltk.corpus import stopwords

#proses stopword
stopword = nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo','kalo', 'amp', 'biar', 'bikin', 'bilang','gak', 'ga', 'krn', 'nya', 
                 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'tai', 'juga', 'utk', 'ya','jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe',
                 'pen', 'u', 'nan', 'loh', 'rt', 'kudu', 'mks', 'wkwk', 'an', 'lah', 'dah', 'asu', 'mjb', 'haha',
                 'yah', 'lha', 'lah', 'hihi', 'ih', 'anjir', 'bgt', 'pake', 'anj', 'bgst', 'anjir'])
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    return df
df['stopword'] = df['tokenisasi'].apply(remove_stopwords)
df.head(2000)

comment  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda-beda harganya?   
2                             ikut nyimak kawan,sukses   
3                                klo ke desa bisa gak?   
4     bang nunggu verivikasi telponnya berapa lama ya?   
5    susah kalau lewat hp.. saya lgsg pergi ke kant...   
6    saya mau pasang tapi tidak ada tiang di depan ...   
33                                           2022 dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya ni saya waktu pasang bayar pert...   
36                    ada paket untuk usaha rt/rw net?   
37                                                 NaN   
38                                 lemes amat suaranya   
67   di apk myindihome ada yang 240rb/ bulan itu as...   
68   mau nanya itu kalau tv dirumahnya msh tv lama ...   
69                      useetv itu nyedot kuota tidak?   
70   kllo pasang indihome baru apakah dapet stb ka?...   
71      kira2 sekitar 200k ada kah di wilayah surabaya   
72                                5 hari yg lalu hadir   
73                  kereeen. salam kenal dan sukses ya   
74                         banyak bacot penjelasannya,   
75   kak kalo dari paket internet dan tv kyk kakak ...   
76   bang, kalo upgrade via aplikasi fup kuotanya n...   
77                                            kga bisa   
78          kalo bisa. minta tolong jangan bisik2 bang   
79   ah ampe 4x nelpon ke 147 ampe skrg ga berubah ...   
80                  kok nambahnya 70 rb yak bukan 20rb   
81   kalau gantipaket/cari lebih murah bisa g bang ...   
88             batas fup 20mbps berapa bang? sama 350?   
89   kok gagal .. nomer layananan anda tidak kompot...   
104  mas saya kok tdk bisa speed on dimaind kenapa ...   
105            upgarad speed sy yg 20mbps 40rebu uuyy😖   
106  saya langganan paket internet 10mb + telepon d...   
107  itu bang selama sudah di upgrade menjadi 20 mb...   
108  saya upgrade 10 mbps 20 mbs apa benner cuma na...   
109  setiap mau upgrade speed dari 10mbps ke 20mbps...   
110               klo 100mbps apa hrs ke plaza telkom?   
111  bang saya kan berlangganan paket phoenix 10 mb...   
112  bang itu speednya k uba setela pembayaran atw ...   
113           bang add on movin itu gratis apa bayar ?   
114  bang kl sy upgrade speed setelah pembayaran bu...   
115                        thanks gan, sangat membantu   
116                    tiap x beli jawaban nya c saldo   

                                        cleaning_tweet  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda beda harganya    
2                             ikut nyimak kawan sukses   
3                                   klo desa bisa gak    
4        bang nunggu verivikasi telponnya berapa lama    
5          susah kalau lewat saya lgsg pergi kantornya   
6    saya mau pasang tapi tidak ada tiang depan rum...   
33                                                dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya saya waktu pasang bayar pertma ...   
36                          ada paket untuk usaha net    
37                                                 nan   
38                                 lemes amat suaranya   
67    apk myindihome ada yang  rb bulan itu asli ng...   
68   mau nanya itu kalau dirumahnya msh lama tabung...   
69                      useetv itu nyedot kuota tidak    
70   kllo pasang indihome baru apakah dapet stb say...   
71           kira  sekitar    ada kah wilayah surabaya   
72                                     hari lalu hadir   
73                     kereeen salam kenal dan sukses    
74                         banyak bacot penjelasannya    
75   kak kalo dari paket internet dan kyk kakak mbp...   
76   bang kalo upgrade via aplikasi fup kuotanya na...   
77                                            kga bisa   
78           kalo bisa minta tolong jangan b

In [10]:
#stemming
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['stopword']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]
    return df

df['stemming'] = df['stopword'].swifter.apply(get_stemmed_term)
df.head(2000)

173
------------------------
bang : bang
daftar : daftar
tingal : tingal
nunggu : nunggu
telepon : telepon
dri : dri
indihome : indihome
trus : trus
batalin : batalin
pemasangan : pasang
 : 
wilayah : wilayah
berbeda : beda
beda : beda
harganya : harga
nyimak : nyimak
kawan : kawan
sukses : sukses
desa : desa
verivikasi : verivikasi
telponnya : telpon
susah : susah
lgsg : lgsg
pergi : pergi
kantornya : kantor
pasang : pasang
tiang : tiang
rumahku : rumah
sebrang : sebrang
jalan : jalan
berjarak : jarak
suruh : suruh
bayar : bayar
juta : juta
gmana : gmana
dunk : dunk
btw : btw
nanya : nanya
indohome : indohome
pemasangnnya : pemasangnnya
pertma : pertma
biaya : biaya
jaminan : jamin
penambhan : penambhan
ppn : ppn
nti : nti
juni : juni
jdinya : jdinya
paket : paket
internet : internet
mbps : mbps
pemasangn : pemasangn
dibilngnya : dibilngnya
nnti : nnti
masuk : masuk
bln : bln
normal : normal
hrus : hrus
usaha : usaha
net : net
lemes : lemes
suaranya : suara
apk : apk
myindihome : myin

Pandas Apply:   0%|          | 0/43 [00:00<?, ?it/s]

comment  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda-beda harganya?   
2                             ikut nyimak kawan,sukses   
3                                klo ke desa bisa gak?   
4     bang nunggu verivikasi telponnya berapa lama ya?   
5    susah kalau lewat hp.. saya lgsg pergi ke kant...   
6    saya mau pasang tapi tidak ada tiang di depan ...   
33                                           2022 dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya ni saya waktu pasang bayar pert...   
36                    ada paket untuk usaha rt/rw net?   
37                                                 NaN   
38                                 lemes amat suaranya   
67   di apk myindihome ada yang 240rb/ bulan itu as...   
68   mau nanya itu kalau tv dirumahnya msh tv lama ...   
69                      useetv itu nyedot kuota tidak?   
70   kllo pasang indihome baru apakah dapet stb ka?...   
71      kira2 sekitar 200k ada kah di wilayah surabaya   
72                                5 hari yg lalu hadir   
73                  kereeen. salam kenal dan sukses ya   
74                         banyak bacot penjelasannya,   
75   kak kalo dari paket internet dan tv kyk kakak ...   
76   bang, kalo upgrade via aplikasi fup kuotanya n...   
77                                            kga bisa   
78          kalo bisa. minta tolong jangan bisik2 bang   
79   ah ampe 4x nelpon ke 147 ampe skrg ga berubah ...   
80                  kok nambahnya 70 rb yak bukan 20rb   
81   kalau gantipaket/cari lebih murah bisa g bang ...   
88             batas fup 20mbps berapa bang? sama 350?   
89   kok gagal .. nomer layananan anda tidak kompot...   
104  mas saya kok tdk bisa speed on dimaind kenapa ...   
105            upgarad speed sy yg 20mbps 40rebu uuyy😖   
106  saya langganan paket internet 10mb + telepon d...   
107  itu bang selama sudah di upgrade menjadi 20 mb...   
108  saya upgrade 10 mbps 20 mbs apa benner cuma na...   
109  setiap mau upgrade speed dari 10mbps ke 20mbps...   
110               klo 100mbps apa hrs ke plaza telkom?   
111  bang saya kan berlangganan paket phoenix 10 mb...   
112  bang itu speednya k uba setela pembayaran atw ...   
113           bang add on movin itu gratis apa bayar ?   
114  bang kl sy upgrade speed setelah pembayaran bu...   
115                        thanks gan, sangat membantu   
116                    tiap x beli jawaban nya c saldo   

                                        cleaning_tweet  \
0    bang kalo misalnya kita sdh daftar tingal nung...   
1            apa setiap wilayah berbeda beda harganya    
2                             ikut nyimak kawan sukses   
3                                   klo desa bisa gak    
4        bang nunggu verivikasi telponnya berapa lama    
5          susah kalau lewat saya lgsg pergi kantornya   
6    saya mau pasang tapi tidak ada tiang depan rum...   
33                                                dunk   
34   bang btw mau nanya pasang indohome itu hargany...   
35   bang mau nanya saya waktu pasang bayar pertma ...   
36                          ada paket untuk usaha net    
37                                                 nan   
38                                 lemes amat suaranya   
67    apk myindihome ada yang  rb bulan itu asli ng...   
68   mau nanya itu kalau dirumahnya msh lama tabung...   
69                      useetv itu nyedot kuota tidak    
70   kllo pasang indihome baru apakah dapet stb say...   
71           kira  sekitar    ada kah wilayah surabaya   
72                                     hari lalu hadir   
73                     kereeen salam kenal dan sukses    
74                         banyak bacot penjelasannya    
75   kak kalo dari paket internet dan kyk kakak mbp...   
76   bang kalo upgrade via aplikasi fup kuotanya na...   
77                                            kga bisa   
78           kalo bisa minta tolong jangan b

In [11]:
Final = df[['stemming']]
Final = Final.rename(columns={'stemming':'Comments'})

In [12]:
Final

Comments
0    [bang, daftar, tingal, nunggu, telepon, dri, i...
1                       [wilayah, beda, beda, harga, ]
2                              [nyimak, kawan, sukses]
3                                             [desa, ]
4                 [bang, nunggu, verivikasi, telpon, ]
5                         [susah, lgsg, pergi, kantor]
6    [pasang, tiang, rumah, tiang, sebrang, jalan, ...
33                                            [, dunk]
34   [bang, btw, nanya, pasang, indohome, harga, ba...
35   [bang, nanya, pasang, bayar, pertma, biaya, ja...
36                               [paket, usaha, net, ]
37                                                  []
38                                      [lemes, suara]
67          [, apk, myindihome, rb, asli, ngga, kak, ]
68       [nanya, rumah, msh, tabung, ngga, flat, baru]
69                           [useetv, nyedot, kuota, ]
70   [kllo, pasang, indihome, dapet, stb, pasang, m...
71                            [kah, wilayah, surabaya]
72                                           [, hadir]
73                   [kereeen, salam, kenal, sukses, ]
74                                    [bacot, jelas, ]
75   [kak, paket, internet, kyk, kakak, mbps, itu, ...
76   [bang, upgrade, via, aplikasi, fup, kuota, nam...
77                                               [kga]
78                               [tolong, bisik, bang]
79   [, ampe, nelpon, ampe, skrg, ubah, mbps, input...
80                                [nambahnya, yak, rb]
81           [gantipaket, cari, murah, bang, indihome]
88                          [batas, fup, mbps, bang, ]
89             [gagal, nomer, layananan, kompotibel, ]
104                         [mas, speed, dimaind, sod]
105               [upgarad, speed, mbps, rebu, uuyy, ]
106  [langgan, paket, internet, mb, telepon, tagih,...
107          [bang, upgrade, mbps, tagih, depan, bang]
108             [upgrade, mbps, mbs, benner, nambah, ]
109  [upgrade, speed, mbps, mbps, mucul, invalid, p...
110                       [mbps, hrs, plaza, telkom, ]
111  [bang, langgan, paket, phoenix, mbps, bayar, u...
112    [bang, speednya, uba, setela, bayar, atw, bang]
113                [bang, add, movin, gratis, bayar, ]
114  [bang, upgrade, speed, bayar, bulan, langsung,...
115                               [thanks, gan, bantu]
116                                      [beli, saldo]

In [13]:
Final.to_excel('databersih_youtube_indihome_1.xlsx')